# Example using Little Alchemy 2 <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs/blob/main/examples/28_little-alchemy_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Little Alchemy 2 is a game, where the goal is to guess recipes to create over 600 elements just using the 4 prime elements air, earth, fire and water.


In this example, we create a small bill of material (BoM) graph using these recipes

Before using the graph widget, install all necessary packages.

In [1]:
%pip install yfiles_jupyter_graphs --quiet
from yfiles_jupyter_graphs import GraphWidget

Note: you may need to restart the kernel to use updated packages.


You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [2]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

<a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs/blob/main/examples/28_little-alchemy_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the library containing all recipe information:

In [3]:
import urllib.request, json
with urllib.request.urlopen("https://unpkg.com/little-alchemy-2@0.0.1/dist/alchemy.json") as url:
    data = json.load(url)

We parse the resulting data to create our nodes and edges.
In the given data, each element has a name and parent information.

We only use a small sample of elements, but you can adjust the constant ```num_elements``` to see more recipes.

In [4]:
import itertools

# we'll use a subset of the graph here
num_elements = 50
dataset = dict(itertools.islice(data.items(), num_elements))

nodes = []
edges = []

for key, item in dataset.items():
    nodes.append({"id": key, 'properties': {'label': item['n']}})
    # for simplicity, we just examine the parents of the items
    if 'p' in item:
        # each element has exactly two (or none) source elements in Little Alchemy 2
        for source1, source2 in item['p']:
            if source1 in dataset and source2 in dataset:
                if not source1 == source2:
                    edges.append({"start": source1, "end": key, "properties": {'label': ('+ ' + data[source2]['n'])}})
                    edges.append({"start": source2, "end": key, "properties": {'label': ('+ ' + data[source1]['n'])}})
                else:
                    edges.append({"start": source1, "end": key, "properties": {'label': ('+ ' + data[source2]['n'])}})

In [5]:
w = GraphWidget()
w.nodes = nodes
w.edges = edges
w.directed = True
display(w)

GraphWidget(layout=Layout(height='800px', width='100%'))

To make the graph easier to read, we add the game images and make the prime elements bigger as well as use the hierarchic layout.

You can check out the other examples like [size mappings](./05_size_mapping.ipynb) and [styles mappings](./08_styles_mapping.ipynb) to learn more about graph customization.

In [6]:
w3 = GraphWidget()
w3.nodes = nodes
w3.edges = edges
w3.directed = True

def custom_images(index, node):
    return {'image': "https://littlealchemy2.com/static/icons/" + node['id'] + ".svg"}

w3.set_node_styles_mapping(custom_images)

def custom_size(index, node):
    if 'prime' in data[str(index+1)]:
        return 80,80
    return 55,55

w3.set_node_size_mapping(custom_size)
w3.hierarchic_layout()
w3.edge_color_mapping = lambda : 'grey'
w3.set_overview(enabled=False)
w3.set_sidebar(enabled=False)
display(w3)

GraphWidget(layout=Layout(height='800px', width='100%'))

You can use the Neighborhood Tab in the sidebar to explore single elements.

However to focus on just one element you're interested in, you can use the following code.

Change the constant ```element_name``` to only see the defined element with its children and parent nodes

In [7]:
# change this to focus on a different element
element_name = 'cat'

nodes2 = []
edges2 = []
parentSet = set()
element_id = None


#This function generates edge labels
def getCombinations(id, target_id):
    result = []
    item = data[target_id]
    for source1, source2 in item['p']:
        if source1 == id:
            result.append(data[source2]['n'])
        if source2 == id:
            result.append(data[source1]['n'])
    return result

for key, item in data.items():
    if item['n'] == element_name:
        nodes2.append({"id": key, 'properties': {'label': item['n']}})
        element_id = key
        if 'p' in item:
            for source1, source2 in item['p']:
                parentSet.add(source1)
                parentSet.add(source2)
        if 'c' in item:
            for child in item['c']:
                if child not in parentSet:
                    nodes2.append({"id":child, 'properties': {'label': data[child]['n']}})
                    edges2.append({"start": key, "end": child, "properties": {'label': '+ ' + str(getCombinations(element_id, child))[1:-1].replace("\'", "")}})
                else:
                    edges2.append({"start": key, "end": child, "properties": {'label': '+ ' + str(getCombinations(element_id, child))[1:-1].replace("\'", "")}})


for source in parentSet:
    if not source == element_id:
        nodes2.append({"id": source, 'properties': {'label': data[source]['n']}})
    edges2.append({"start": source, "end": element_id, "properties": {'label': '+ ' + str(getCombinations(source, element_id))[1:-1].replace("\'", "")}})


We use some of the mappings above:

- Game image node styling
- directed, grey edges
- hierarchic layout


In [8]:
w2 = GraphWidget()
w2.nodes = nodes2
w2.edges = edges2
w2.set_node_styles_mapping(custom_images)
w2.directed = True
w2.hierarchic_layout()
w2.edge_color_mapping = lambda : 'grey'
display(w2)

GraphWidget(layout=Layout(height='500px', width='100%'))